In [1]:
*** Mastering 'Metrics
*** Generates Table 3.3
*** Minneapolis Domestic Violence Experiment
*** Created by Jon Petkun (jbpetkun@mit.edu) on Feb 12, 2015

version 12
clear all

In [11]:
cd "/Users/datalab/bigdata/mastering_metrics_data"

/Users/datalab/bigdata/mastering_metrics_data


In [12]:
ls


total 202232
-rw-r--r--@ 1 datalab  staff  36844584  2  3  2013 AA_small.dta
-rw-r--r--@ 1 datalab  staff   6637632  3  4 00:46 AA_small.dta_.zip
-rw-r--r--@ 1 datalab  staff      7697  3  4 00:45 AEJfigs.dta
-rw-r--r--@ 1 datalab  staff   3490484  3  4 00:42 Data1.zip
drwxr-xr-x@ 6 datalab  staff       192  3  4 00:35 NHIS/
drwxr-xr-x@ 9 datalab  staff       288  3  4 13:01 OHIE_Public_Use_Files/
-rw-r--r--@ 1 datalab  staff       604  3  4 00:36 ReadMe_RAND.txt
-rw-r--r--@ 1 datalab  staff   6591291  3  4 00:46 ak91.dta
-rw-r--r--@ 1 datalab  staff   1006144  1 22  2015 annual_spend.dta
-rw-r--r--@ 1 datalab  staff     77216  3  4 00:45 banks.csv
-rw-r--r--@ 1 datalab  staff      2481  3  4 00:46 clark_martorell_cellmeans.dta
-rw-r--r--@ 1 datalab  staff   1168671  3  4 00:45 deaths.dta
-rw-r--r--@ 1 datalab  staff     29249  3  4 00:45 mdve.dta
-rw-r--r--@ 1 datalab  staff   9370270  3  4 00:45 oregon_puf.zip
-rw-r--r--@ 1 datalab  staff  27605676  3 23  2014 person_years.dta
-rw-r

In [13]:
use mdve.dta, clear


In [14]:
* Generate action assignments (i.e. what are police assigned to do)
gen z_arrest=(T_RANDOM == 1)
gen z_advise=(T_RANDOM == 2)
gen z_separate=(T_RANDOM == 3)

In [15]:
* Generate actual outcomes (i.e. what action do the police actually take)
gen d_arrest=(T_FINAL == 1)
gen d_advise=(T_FINAL == 2)
gen d_separate=(T_FINAL == 3)
gen d_other=(T_FINAL == 4)
gen z_coddled=(z_arrest == 0)
gen d_coddled=(d_arrest == 0)
gen assigned = "Arrest" if T_RANDOM == 1
replace assigned = "Advise" if T_RANDOM == 2
replace assigned = "Separate" if T_RANDOM == 3
gen outcome = "Arrest" if T_FINAL == 1
replace outcome = "Advise" if T_FINAL == 2
replace outcome = "Separate" if T_FINAL == 3
replace outcome = "Other" if T_FINAL == 4
gen total = 1








(237 missing values generated)

(110 real changes made)

variable assigned was str6 now str8
(127 real changes made)

(194 missing values generated)

(89 real changes made)

variable outcome was str6 now str8
(89 real changes made)

(16 real changes made)



In [17]:
*ssc install estout

checking estout consistency and verifying not already installed...
installing into /Users/datalab/Library/Application Support/Stata/ado/plus/...
installation complete.


In [18]:
* Drop if actual outcome is "other"
keep if d_other == 0

* Tabulate assignments and outcomes
estpost tabstat d_arrest d_advise d_separate total, by(assigned) listwise ///
    statistics(sum  mean) columns(statistics)


(0 observations deleted)


Summary statistics: sum mean
     for variables: d_arrest d_advise d_separate total
  by categories of: assigned

    assigned |    e(sum)    e(mean) 
-------------+----------------------
Advise       |                      
    d_arrest |        19   .1759259 
    d_advise |        84   .7777778 
  d_separate |         5   .0462963 
       total |       108          1 
-------------+----------------------
Arrest       |                      
    d_arrest |        91   .9891304 
    d_advise |         0          0 
  d_separate |         1   .0108696 
       total |        92          1 
-------------+----------------------
Separate     |                      
    d_arrest |        26   .2280702 
    d_advise |         5   .0438596 
  d_separate |        83   .7280702 
       total |       114          1 
-------------+----------------------
Total        |                      
    d_arrest |       136    .433121 
    d_advise |        89   .2834395 
  d_sep

In [19]:
* Insert tabulations into matrix	
matrix S = e(sum)
matrix T = J(4,8,.)
mat T[1,2] = S[1,5]
mat T[1,4] = S[1,6]
mat T[1,6] = S[1,7]
mat T[2,2] = S[1,1]
mat T[2,4] = S[1,2]
mat T[2,6] = S[1,3]
mat T[3,2] = S[1,9]
mat T[3,4] = S[1,10]
mat T[3,6] = S[1,11]
mat T[1,8] = T[1,2]+T[1,4]+T[1,6]
mat T[2,8] = T[2,2]+T[2,4]+T[2,6]
mat T[3,8] = T[3,2]+T[3,4]+T[3,6]
mat T[4,2] = T[1,2]+T[2,2]+T[3,2]
mat T[4,4] = T[1,4]+T[2,4]+T[3,4]
mat T[4,6] = T[1,6]+T[2,6]+T[3,6]
mat T[4,8] = T[1,8]+T[2,8]+T[3,8]

In [20]:
* Generate percentage values
forval i = 1/4 {
forval j = 1/3 {
local k = `j'*2
local r = `k' - 1
mat T[`i',`r'] = 100*T[`i',`k']/T[`i',8]
}
mat T[`i',7] = 100*T[`i',8]/T[4,8]
}

In [22]:
*ssc install mat2txt

checking mat2txt consistency and verifying not already installed...
installing into /Users/datalab/Library/Application Support/Stata/ado/plus/...
installation complete.


In [27]:
* Row and column labels
matrix rownames T = Arrest Advise Separate Total
matrix colnames T = Arrest Arrest Advise Advise Separate Separate Total Total

mat2txt, matrix(T) saving("/Users/datalab/github/datascience/causal-inference/Table33.xls") replace
